In [1]:
import torch
import matplotlib.pyplot as plt
import os
import sys
import warnings
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from TF_net.train import Dataset, train_epoch, eval_epoch, test_epoch, Scaler, preprocess
import torch.nn as nn
from torch.utils import data
from pathlib import Path
import numpy as np

class Args:
    pass

/home/lakshya/data/miniconda3/envs/tfnet/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = Args() 
args.data = 'data5.pt'


if args.data == "rbc_data.pt":
    compress = True
    permute = False
    transform_type = 'std'
elif args.data == 'data5.pt':
    compress = False
    permute = True
    transform_type = 'norm'
else:
    raise ValueError("Un expected data file name")

if Path(args.data).name == args.data: args.data = "../" + args.data

args.transform = Scaler(transform_type)
data_prep = preprocess(args, permute, compress, test_mode=True)

tensor([-224.2000])
tensor([723.7476])


In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

def norm_each_sample(x):
    x = torch.clone(x)
    _max= torch.amax(x, dim=(2, 3), keepdim=True) 
    _min= torch.amin(x, dim=(2, 3), keepdim=True) 
    return (x - _min) / (_max - _min)

def plot(video, channel, frames = None):
    if frames is None:
        frames = video.shape[0]
    warnings.warn("Going to normalize each sample for visualizing purpose")
    video = norm_each_sample(video)
    fig = plt.figure()
    im = plt.imshow(video[0,channel,:,:])

    plt.close() # this is required to not display the generated image

    def init():
        im.set_data(video[0,channel,:,:])

    def animate(i):
        im.set_data(video[i,channel,:,:])
        return im

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=frames,
                                interval=50)
    return anim.to_html5_video()

HTML(plot(data_prep, 0, 120))

In [1]:
test_indices = list(range(7700, 9800))
input_length = 25
time_range = 6
batch_size = 21
device = "cuda:"
device_ids = 7
device = torch.device(f"cuda:{device_ids[0]}" if torch.cuda.is_available() else "cpu")
loss_fun = torch.nn.MSELoss()
best_model = nn.DataParallel(torch.load(args.path+"model.pth", map_location=device).module, device_ids=device_ids)
data_prep = preprocess(args, permute, compress, test_mode=True)

# On test set
if not args.only_val:
    print("Testing in test setting")
    test_set = Dataset(test_indices, input_length + time_range - 1, 40, 60, data_prep, True,test_mode=True)
    test_loader = data.DataLoader(test_set, batch_size = batch_size, shuffle = False, num_workers = 8)
    preds, trues, loss_curve = test_epoch(args, test_loader, best_model, loss_fun, test_mode=True,device=device)

    torch.save({"loss_curve": loss_curve}, 
                args.path+"results.pt",pickle_protocol=5)

NameError: name 'torch' is not defined